In [ ]:
import ee, geemap
ee.Initialize(project='ee-krle4401')

aoi = ee.Geometry.Rectangle([121.26, 31.06, 121.39, 31.15])
capella_date = ee.Date('2025-03-20')  # from your JSON

# Search ± 6 months for HH in any mode
window_days = 180
s1_hh_any = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate(capella_date.advance(-window_days, 'day'),
                capella_date.advance(window_days, 'day'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
)

# Sort by temporal distance to your Capella date
def add_time_diff(img):
    diff = img.date().difference(capella_date, 'day').abs()
    return img.set('time_diff_days', diff)

s1_hh_sorted = s1_hh_any.map(add_time_diff).sort('time_diff_days')

closest_hh = ee.Image(s1_hh_sorted.first())
print("Closest HH scene date:", closest_hh.date().format().getInfo())

# Visualize
vis = {'bands':['HH'], 'min': -25, 'max': 0}
Map = geemap.Map(center=[31.105, 121.325], zoom=12)
Map.addLayer(closest_hh.select('HH'), vis, 'Closest Sentinel-1 HH (dB)')
Map.addLayer(aoi, {}, 'AOI')
Map


HH scenes (any mode): 0
HH scenes (IW only): 0


Map(center=[31.105, 121.325], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…

#### Get GBIF data for Shanghai in 2024 and 2025